In [52]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import timm
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

import matplotlib.image as mpimg

In [53]:
# === CONFIG ===
IMG_SIZE = 224 
BATCH_SIZE = 32
EPOCHS = 3
LR = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0))
print("Se folosește:", DEVICE)

NVIDIA GeForce RTX 3050 Laptop GPU
Se folosește: cuda


In [54]:
# Recrearea arhitecturii modelului
model = timm.create_model('vit_small_patch16_224', pretrained=False, num_classes=2)
model.head = nn.Linear(model.head.in_features, 2)  # Asigură-te că e identic cu modelul salvat

# Încarcare parametrii
model.load_state_dict(torch.load('mamografiiPro.pth', map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
model.to(DEVICE)
model.eval()  # setează în modul de inferență

C:\Users\Claudiu\AppData\Local\Temp\ipykernel_29196\2608548833.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('mamografiiPro.pth', map_

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [55]:
transform = transforms.Compose([
		transforms.RandomHorizontalFlip(),
		transforms.RandomRotation(10),
		transforms.ColorJitter(brightness=0.1, contrast=0.1),
		transforms.Resize((IMG_SIZE, IMG_SIZE)),
		transforms.ToTensor(),
		transforms.Normalize([0.5]*3, [0.5]*3)
	])

img_path = "test\\16896\\1\\16896_idx5_x701_y1451_class1.png"
image = Image.open(img_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0).to(DEVICE) 

with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)
    print("Predicted class:", predicted.item())

class_map = {0: "no tumor", 1: "tumor"}
print("Result:", class_map[predicted.item()])

Predicted class: 1
Result: tumor


In [56]:
# Recrearea arhitecturii modelului
model2 = timm.create_model('vit_small_patch16_224', pretrained=False, num_classes=2)
model2.head = nn.Linear(model2.head.in_features, 2)  # Asigură-te că e identic cu modelul salvat

# Încarcare parametrii
model2.load_state_dict(torch.load('mamografii.pth', map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
model2.to(DEVICE)
model2.eval()  # setează în modul de inferență

C:\Users\Claudiu\AppData\Local\Temp\ipykernel_29196\79433350.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load('mamografii.pth', map_loca

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [57]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),         # Resize to match model input
    transforms.ToTensor(),                 # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # Match training normalization
])

img_path = "test\\16896\\1\\16896_idx5_x701_y1451_class1.png"
image = Image.open(img_path).convert("RGB")
input_tensor = transform(image).unsqueeze(0).to(DEVICE) 

with torch.no_grad():
    output = model2(input_tensor)
    _, predicted = torch.max(output, 1)
    print("Predicted class:", predicted.item())

class_map = {0: "no tumor", 1: "tumor"}
print("Result:", class_map[predicted.item()])

Predicted class: 0
Result: no tumor
